    install python 3.8.x  &  jupyter notebook
    pip install --upgrade gensim
    pip3 install NlpToolkit-Corpus
    pip install pandas

<!--
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires protobuf>=3.9.2, which is not installed.
tensorflow 2.5.0 requires typing-extensions~=3.7.4, which is not installed.
tensorflow 2.5.0 requires wrapt~=1.12.1, which is not installed.
tensorboard 2.5.0 requires google-auth<2,>=1.6.3, which is not installed.
tensorboard 2.5.0 requires protobuf>=3.6.0, which is not installed.
tensorboard 2.5.0 requires requests<3,>=2.21.0, which is not installed.
tensorboard 2.5.0 requires tensorboard-data-server<0.7.0,>=0.6.0, which is not installed.
tensorboard 2.5.0 requires tensorboard-plugin-wit>=1.6.0, which is not installed.
tensorboard 2.5.0 requires werkzeug>=0.11.15, which is not installed.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.2 which is incompatible.
tensorflow 2.5.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
-->

In [8]:
TRAIN_MODEL = False # if false: load model, else train the model (which takes 70 minutes @ Intel i7-10750H - 12*2.60GHz)

In [1]:
from Corpus.TurkishSplitter import TurkishSplitter
import gensim
import logging
import multiprocessing
import os
import re
import sys
import pandas as pd
import pickle # because mine was not python=3.8.x
import numpy as np

#from pattern.en import tokenize
from time import time

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

C:\Users\aeunal\anaconda3\envs\py38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
data_path = "../data/merged.gz"
data = pd.read_pickle(data_path, compression="gzip")
data = data.fillna(" ")

In [3]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

class MySentences(object):
    def __init__(self, data_path):
        self.data_path = data_path
        data = pd.read_pickle(data_path, compression="gzip")
        self.data = data.fillna(" ")

    def __iter__(self):
        for text in self.data:
            cleaned_text = cleanhtml(text)
            for sentence in TurkishSplitter().split(cleaned_text):
                sentence = str(sentence)
                for line in sentence.split("\n"):
                    sline = line.strip()
                    if sline == "":
                        continue
                    #tokenized_line = ' '.join(tokenize(rline))
                    is_alpha_word_line = [word for word in
                                          sline.lower().split()
                                          if word.isalpha()]
                    yield is_alpha_word_line

In [7]:
if not TRAIN_MODEL:
    model = gensim.models.Word2Vec.load("D:/Git/NLHack/data/word2vec_model/word2vec_gensim")

2021-08-19 12:29:25,613 : INFO : loading Word2Vec object from D:/Git/NLHack/data/word2vec_model/word2vec_gensim
2021-08-19 12:29:25,685 : INFO : loading wv recursively from D:/Git/NLHack/data/word2vec_model/word2vec_gensim.wv.* with mmap=None
2021-08-19 12:29:25,686 : INFO : loading vectors from D:/Git/NLHack/data/word2vec_model/word2vec_gensim.wv.vectors.npy with mmap=None
2021-08-19 12:29:27,549 : INFO : Word2Vec lifecycle event {'fname': 'D:/Git/NLHack/data/word2vec_model/word2vec_gensim', 'datetime': '2021-08-19T12:29:27.549152', 'gensim': '4.0.1', 'python': '3.8.11 (default, Aug  6 2021, 09:57:55) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'loaded'}


In [10]:
if not TRAIN_MODEL:
    model.wv.load_word2vec_format("D:/Git/NLHack/data/word2vec_model/word2vec_org",
                                  "D:/Git/NLHack/data/word2vec_model/vocabulary",
                                  binary=False)

2021-08-19 12:34:12,885 : INFO : loading word counts from D:/Git/NLHack/data/word2vec_model/vocabulary
2021-08-19 12:34:13,174 : INFO : loading projection weights from D:/Git/NLHack/data/word2vec_model/word2vec_org
2021-08-19 12:34:37,217 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (224492, 200) matrix of type float32 from D:/Git/NLHack/data/word2vec_model/word2vec_org', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-08-19T12:34:37.217754', 'gensim': '4.0.1', 'python': '3.8.11 (default, Aug  6 2021, 09:57:55) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'load_word2vec_format'}


In [50]:
if TRAIN_MODEL:
    begin = time()

    sentences = MySentences(data_path)
    model = gensim.models.Word2Vec(sentences,
                                   vector_size=200,
                                   window=10,
                                   min_count=10,
                                   workers=multiprocessing.cpu_count())

    end = time()
    print("Total procesing time: %d seconds" % (end - begin))

2021-08-17 19:54:56,070 : INFO : collecting all words and their counts
2021-08-17 19:54:56,080 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-17 19:54:57,476 : INFO : PROGRESS: at sentence #10000, processed 134360 words, keeping 29427 word types
2021-08-17 19:54:58,680 : INFO : PROGRESS: at sentence #20000, processed 245470 words, keeping 45890 word types
2021-08-17 19:54:59,704 : INFO : PROGRESS: at sentence #30000, processed 341326 words, keeping 58033 word types
2021-08-17 19:55:01,161 : INFO : PROGRESS: at sentence #40000, processed 476378 words, keeping 68533 word types
2021-08-17 19:55:02,704 : INFO : PROGRESS: at sentence #50000, processed 617719 words, keeping 76927 word types
2021-08-17 19:55:04,198 : INFO : PROGRESS: at sentence #60000, processed 761005 words, keeping 84774 word types
2021-08-17 19:55:05,696 : INFO : PROGRESS: at sentence #70000, processed 904119 words, keeping 91412 word types
2021-08-17 19:55:07,437 : INFO : PROGRESS: at 

2021-08-17 19:56:31,274 : INFO : PROGRESS: at sentence #710000, processed 8598957 words, keeping 285337 word types
2021-08-17 19:56:33,093 : INFO : PROGRESS: at sentence #720000, processed 8753529 words, keeping 287105 word types
2021-08-17 19:56:34,979 : INFO : PROGRESS: at sentence #730000, processed 8917143 words, keeping 288784 word types
2021-08-17 19:56:36,240 : INFO : PROGRESS: at sentence #740000, processed 9032905 words, keeping 290787 word types
2021-08-17 19:56:37,142 : INFO : PROGRESS: at sentence #750000, processed 9119433 words, keeping 294374 word types
2021-08-17 19:56:38,120 : INFO : PROGRESS: at sentence #760000, processed 9206070 words, keeping 297630 word types
2021-08-17 19:56:39,197 : INFO : PROGRESS: at sentence #770000, processed 9304922 words, keeping 301939 word types
2021-08-17 19:56:40,342 : INFO : PROGRESS: at sentence #780000, processed 9399962 words, keeping 306059 word types
2021-08-17 19:56:41,374 : INFO : PROGRESS: at sentence #790000, processed 949207

2021-08-17 19:58:04,006 : INFO : PROGRESS: at sentence #1420000, processed 16928152 words, keeping 472818 word types
2021-08-17 19:58:05,251 : INFO : PROGRESS: at sentence #1430000, processed 17046470 words, keeping 474797 word types
2021-08-17 19:58:06,485 : INFO : PROGRESS: at sentence #1440000, processed 17164321 words, keeping 476695 word types
2021-08-17 19:58:07,758 : INFO : PROGRESS: at sentence #1450000, processed 17282192 words, keeping 478626 word types
2021-08-17 19:58:08,999 : INFO : PROGRESS: at sentence #1460000, processed 17399581 words, keeping 480669 word types
2021-08-17 19:58:10,224 : INFO : PROGRESS: at sentence #1470000, processed 17517022 words, keeping 482626 word types
2021-08-17 19:58:11,481 : INFO : PROGRESS: at sentence #1480000, processed 17635923 words, keeping 484617 word types
2021-08-17 19:58:12,746 : INFO : PROGRESS: at sentence #1490000, processed 17755397 words, keeping 486573 word types
2021-08-17 19:58:14,012 : INFO : PROGRESS: at sentence #1500000,

2021-08-17 19:59:40,983 : INFO : PROGRESS: at sentence #2130000, processed 26057624 words, keeping 699087 word types
2021-08-17 19:59:42,409 : INFO : PROGRESS: at sentence #2140000, processed 26191050 words, keeping 701558 word types
2021-08-17 19:59:43,844 : INFO : PROGRESS: at sentence #2150000, processed 26321879 words, keeping 704723 word types
2021-08-17 19:59:45,189 : INFO : PROGRESS: at sentence #2160000, processed 26452410 words, keeping 707609 word types
2021-08-17 19:59:46,517 : INFO : PROGRESS: at sentence #2170000, processed 26582367 words, keeping 710857 word types
2021-08-17 19:59:47,862 : INFO : PROGRESS: at sentence #2180000, processed 26714079 words, keeping 714132 word types
2021-08-17 19:59:49,272 : INFO : PROGRESS: at sentence #2190000, processed 26849249 words, keeping 717022 word types
2021-08-17 19:59:50,673 : INFO : PROGRESS: at sentence #2200000, processed 26979087 words, keeping 719915 word types
2021-08-17 19:59:52,160 : INFO : PROGRESS: at sentence #2210000,

2021-08-17 20:01:06,333 : INFO : PROGRESS: at sentence #2840000, processed 34030452 words, keeping 859961 word types
2021-08-17 20:01:07,681 : INFO : PROGRESS: at sentence #2850000, processed 34151068 words, keeping 862162 word types
2021-08-17 20:01:09,076 : INFO : PROGRESS: at sentence #2860000, processed 34274087 words, keeping 864775 word types
2021-08-17 20:01:10,504 : INFO : PROGRESS: at sentence #2870000, processed 34401065 words, keeping 867229 word types
2021-08-17 20:01:11,901 : INFO : PROGRESS: at sentence #2880000, processed 34523896 words, keeping 869296 word types
2021-08-17 20:01:13,194 : INFO : PROGRESS: at sentence #2890000, processed 34639240 words, keeping 871334 word types
2021-08-17 20:01:14,408 : INFO : PROGRESS: at sentence #2900000, processed 34749295 words, keeping 873628 word types
2021-08-17 20:01:15,715 : INFO : PROGRESS: at sentence #2910000, processed 34866448 words, keeping 875972 word types
2021-08-17 20:01:16,943 : INFO : PROGRESS: at sentence #2920000,

...

2021-08-17 21:01:07,263 : INFO : EPOCH 5 - PROGRESS: at 60.77% examples, 81268 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:08,322 : INFO : EPOCH 5 - PROGRESS: at 60.93% examples, 81275 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:09,414 : INFO : EPOCH 5 - PROGRESS: at 61.07% examples, 81277 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:10,454 : INFO : EPOCH 5 - PROGRESS: at 61.21% examples, 81268 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:11,550 : INFO : EPOCH 5 - PROGRESS: at 61.36% examples, 81269 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:12,657 : INFO : EPOCH 5 - PROGRESS: at 61.51% examples, 81269 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:13,752 : INFO : EPOCH 5 - PROGRESS: at 61.66% examples, 81270 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:14,828 : INFO : EPOCH 5 - PROGRESS: at 61.81% examples, 81274 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:01:15,894 : INFO : EPOCH 5 - PROGRESS: at 61.94% examples, 81259 words/s, in_qsize 0, out_

2021-08-17 21:02:26,449 : INFO : EPOCH 5 - PROGRESS: at 71.45% examples, 81102 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:27,562 : INFO : EPOCH 5 - PROGRESS: at 71.60% examples, 81100 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:28,686 : INFO : EPOCH 5 - PROGRESS: at 71.75% examples, 81097 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:29,761 : INFO : EPOCH 5 - PROGRESS: at 71.90% examples, 81100 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:30,775 : INFO : EPOCH 5 - PROGRESS: at 72.04% examples, 81097 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:31,793 : INFO : EPOCH 5 - PROGRESS: at 72.17% examples, 81093 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:32,916 : INFO : EPOCH 5 - PROGRESS: at 72.31% examples, 81090 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:33,920 : INFO : EPOCH 5 - PROGRESS: at 72.44% examples, 81089 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:02:35,027 : INFO : EPOCH 5 - PROGRESS: at 72.60% examples, 81088 words/s, in_qsize 0, out_

2021-08-17 21:03:45,325 : INFO : EPOCH 5 - PROGRESS: at 82.36% examples, 80942 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:46,399 : INFO : EPOCH 5 - PROGRESS: at 82.51% examples, 80947 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:47,462 : INFO : EPOCH 5 - PROGRESS: at 82.64% examples, 80938 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:48,480 : INFO : EPOCH 5 - PROGRESS: at 82.78% examples, 80936 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:49,488 : INFO : EPOCH 5 - PROGRESS: at 82.92% examples, 80934 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:50,600 : INFO : EPOCH 5 - PROGRESS: at 83.08% examples, 80935 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:51,676 : INFO : EPOCH 5 - PROGRESS: at 83.24% examples, 80939 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:52,705 : INFO : EPOCH 5 - PROGRESS: at 83.37% examples, 80935 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:03:53,821 : INFO : EPOCH 5 - PROGRESS: at 83.53% examples, 80934 words/s, in_qsize 0, out_

2021-08-17 21:05:03,665 : INFO : EPOCH 5 - PROGRESS: at 93.01% examples, 80865 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:04,707 : INFO : EPOCH 5 - PROGRESS: at 93.15% examples, 80857 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:05,748 : INFO : EPOCH 5 - PROGRESS: at 93.28% examples, 80854 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:06,748 : INFO : EPOCH 5 - PROGRESS: at 93.42% examples, 80855 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:07,778 : INFO : EPOCH 5 - PROGRESS: at 93.56% examples, 80851 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:08,856 : INFO : EPOCH 5 - PROGRESS: at 93.71% examples, 80856 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:09,865 : INFO : EPOCH 5 - PROGRESS: at 93.86% examples, 80855 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:10,896 : INFO : EPOCH 5 - PROGRESS: at 93.99% examples, 80851 words/s, in_qsize 0, out_qsize 0
2021-08-17 21:05:11,994 : INFO : EPOCH 5 - PROGRESS: at 94.15% examples, 80853 words/s, in_qsize 0, out_

FileNotFoundError: [Errno 2] No such file or directory: 'data/model/word2vec_gensim.wv.vectors.npy'

In [54]:
if TRAIN_MODEL:
    model.save("D:/Git/NLHack/data/word2vec_model/word2vec_gensim")

2021-08-17 23:33:48,486 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'D:/Git/NLHack/data/model/word2vec_gensim', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-08-17T23:33:48.486673', 'gensim': '4.0.1', 'python': '3.8.11 (default, Aug  6 2021, 09:57:55) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'saving'}
2021-08-17 23:33:48,486 : INFO : storing np array 'vectors' to D:/Git/NLHack/data/model/word2vec_gensim.wv.vectors.npy
2021-08-17 23:33:49,015 : INFO : saved D:/Git/NLHack/data/model/word2vec_gensim


In [55]:
if TRAIN_MODEL:
    model.wv.save_word2vec_format("D:/Git/NLHack/data/word2vec_model/word2vec_org",
                                  "D:/Git/NLHack/data/word2vec_model/vocabulary",
                                  binary=False)

2021-08-17 23:34:04,095 : INFO : storing vocabulary in D:/Git/NLHack/data/model/vocabulary
2021-08-17 23:34:04,372 : INFO : storing 224492x200 projection weights into D:/Git/NLHack/data/model/word2vec_org


### Only use words with lower characters to query embedded vectors

#### We get (200,1) sized vectors for 1.344.041 unique Turkish (and some English) words 

In [25]:
model.wv["Türkçe".lower()]

array([ 2.7241940e+00, -3.2462633e-01,  1.1839889e+00, -1.6206272e+00,
        4.7765216e-01,  6.8595016e-01,  6.5332073e-01,  1.0332417e+00,
        1.2423421e+00, -1.5151964e+00,  2.5467196e-01, -8.8017458e-01,
       -1.6477575e+00, -9.4083488e-01,  3.2264123e+00, -1.7666215e+00,
        4.4854492e-01, -1.1490281e+00, -2.1297736e+00, -6.9610220e-01,
        1.4873213e+00,  9.8333806e-01,  1.6681736e+00,  2.2539523e+00,
        1.1922356e+00, -1.2609200e+00, -1.6739720e+00,  2.8418818e-02,
       -2.6783164e+00,  1.2456909e+00, -2.6049489e-01, -2.4445327e-01,
        1.9108915e+00, -7.5412661e-01,  1.8114163e+00, -1.8500021e+00,
        5.8091867e-01, -3.2197921e+00, -1.0919235e+00,  2.8463521e+00,
        1.0142922e+00, -8.3994311e-01, -7.2973835e-01, -1.3792421e+00,
       -5.8123833e-01,  1.2617069e+00, -4.5801899e-01,  1.1999007e+00,
        1.5558444e+00,  1.5521458e+00,  1.3657665e+00,  4.1162462e+00,
        3.9536697e-01,  2.8790540e-01, -7.7949566e-01, -3.5304135e-01,
      

In [45]:
model.wv.most_similar("portakal", topn=30)

[('limon', 0.6705577969551086),
 ('mandalina', 0.6473368406295776),
 ('koza', 0.6388903856277466),
 ('ahududu', 0.6352425217628479),
 ('çilek', 0.632841944694519),
 ('sarısı', 0.6211673617362976),
 ('nar', 0.6165995001792908),
 ('palmiye', 0.6108548045158386),
 ('tepside', 0.599723756313324),
 ('nane', 0.5986536741256714),
 ('turunç', 0.5924507975578308),
 ('böğürtlen', 0.586935818195343),
 ('bamya', 0.5832722783088684),
 ('ekşisi', 0.5813543200492859),
 ('kişniş', 0.5810949802398682),
 ('tarçın', 0.5804669260978699),
 ('bakla', 0.5791761875152588),
 ('rezene', 0.5778511166572571),
 ('hızma', 0.5750309824943542),
 ('liçi', 0.5733539462089539),
 ('ananas', 0.572425127029419),
 ('enginar', 0.5712932348251343),
 ('armudu', 0.5709013342857361),
 ('dut', 0.5697391629219055),
 ('bergamot', 0.5694823861122131),
 ('avakado', 0.5659313201904297),
 ('kekik', 0.5657211542129517),
 ('badem', 0.5656008124351501),
 ('sarısıdır', 0.5651161074638367),
 ('reçeli', 0.564294159412384)]

In [29]:
# Search with words
model.wv.most_similar(positive=['kadın', 'kral'], negative=['adam'])

[('kralın', 0.5198776721954346),
 ('kraliçe', 0.5020248293876648),
 ('krallığın', 0.476032555103302),
 ('papalık', 0.47033101320266724),
 ('hanedanın', 0.4617766737937927),
 ('kralı', 0.45486149191856384),
 ('erkek', 0.44943681359291077),
 ('krallığı', 0.4382547438144684),
 ('imparatorun', 0.43367570638656616),
 ('kudüs', 0.42927610874176025)]

In [35]:
# Search with vectors 
adam_kadin = model.wv["adam"] - model.wv["kadın"]
model.wv.most_similar(positive=[adam_kadin])
# same as => model.wv.similar_by_vector(adam_kadin)

[('adam', 0.5862725973129272),
 ('direktörlüğünde', 0.5099519491195679),
 ('direktör', 0.4785635769367218),
 ('direktörlerinden', 0.4748399555683136),
 ('direktörümüz', 0.46246111392974854),
 ('direktöre', 0.4618333578109741),
 ('direktörlerinin', 0.45027175545692444),
 ('direktörlükte', 0.4496172368526459),
 ('direktörün', 0.4464806318283081),
 ('direktörlerin', 0.436387836933136)]

In [58]:
print("Word frequencies of the dataset")
get_first_word_count = 100
for index, word in enumerate(model.wv.index_to_key):
    if index == get_first_word_count:
        break
    print(f"{word:8}\t: {model.wv.get_vecattr(word,'count')}")

Word frequencies of the dataset
ve      	: 1953705
bir     	: 1307763
bu      	: 689482
ile     	: 456658
olarak  	: 440591
da      	: 404264
için    	: 366020
de      	: 356932
olan    	: 291842
daha    	: 227445
sonra   	: 224970
yılında 	: 222450
en      	: 221000
tarafından	: 205544
çok     	: 203298
ise     	: 187969
ilk     	: 174511
gibi    	: 161718
büyük   	: 144370
yer     	: 143002
kadar   	: 141580
vardır  	: 125217
ancak   	: 118936
arasında	: 116249
iki     	: 112890
her     	: 109977
göre    	: 107874
olduğu  	: 105086
yeni    	: 105043
oldu    	: 104440
o       	: 93756
aynı    	: 91280
son     	: 88512
yoktur  	: 88267
veya    	: 85420
diğer   	: 84641
türk    	: 83847
ya      	: 82908
alan    	: 80391
tarihinde	: 79233
birlikte	: 79042
yıl     	: 78631
ama     	: 78206
olduğunu	: 76006
önemli  	: 74316
bağlı   	: 74315
bulunan 	: 74068
genel   	: 71660
önce    	: 71418
ayrıca  	: 70556
yapılan 	: 69773
türkiye 	: 68774
karşı   	: 67375
etti    	: 66979
içinde  	: 6577

In [36]:
v = lambda word: model.wv[word.lower()]

In [43]:
print("Distance Between:")
print("Kral-Adam\t",     np.linalg.norm(v("Kral")-v("Adam")))
print("Kraliçe-Kadın\t", np.linalg.norm(v("Kraliçe")-v("Kadın")))
print("Kraliçe-Kadın\t", np.linalg.norm(v("Kraliçe")-v("Kadın")))

Distance Between:
Kral-Adam	 28.766603
Kraliçe-Kadın	 25.395077
